<a href="https://colab.research.google.com/github/CruiseBott/Prodigytech_Tasks/blob/main/Task1/Task1(TextGeneration_using_GPT2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mounting drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
# Step 2: Load Pre-trained GPT-2 Model and Tokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

model_name = "gpt2"  # You can also use "gpt2-medium", "gpt2-large", etc.
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Step 3: Load and Tokenize the Dataset
def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128
    )

train_dataset = load_dataset('/content/drive/MyDrive/Colab Notebooks/preprocessed_chat_data.txt', tokenizer)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# Step 4: Prepare Data for Training
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


In [ ]:
!pip install transformers[torch]

In [ ]:
# Step 5: Set Up Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)


In [ ]:
# Step 6: Initialize Trainer and Start Training
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=219, training_loss=2.605347515785531, metrics={'train_runtime': 56.0211, 'train_samples_per_second': 15.583, 'train_steps_per_second': 3.909, 'total_flos': 57026985984000.0, 'train_loss': 2.605347515785531, 'epoch': 3.0})

In [ ]:
# Step 7: Save the Fine-Tuned Model
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/fine_tuned_model")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/fine_tuned_model")

('/content/drive/MyDrive/Colab Notebooks/fine_tuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/fine_tuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/fine_tuned_model/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/fine_tuned_model/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/fine_tuned_model/added_tokens.json')

In [ ]:
# Step 8: Generate Text with the Fine-Tuned Model
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Colab Notebooks/fine_tuned_model")
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/fine_tuned_model")

prompt = "I am a vegan and i am in the midwest"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(
    inputs["input_ids"],
    max_length=50,  # Adjust this value as needed
    num_return_sequences=1,
    no_repeat_ngram_size=2,  # Helps to avoid repetition
    early_stopping=True  # Stop generation when it is likely to have finished
)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
print(generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I am a vegan and i am in the midwest.
I like to eat healthy food
Do you like animals?
Yes, i do. i like it. but i have to go to the vet
What is your favorite food?
